In [ ]:
'''
代理可能不可靠，并可能需要人工输入才能成功完成任务。类似地，对于某些操作，您可能希望在运行之前要求人工批准
以确保一切按预期运行

LangGraph 的持久化层支持人工介入工作流，允许根据用户反馈暂停和恢复执行。此功能的主要接口是 interrupt 函数
在节点内部调用 interrupt 将暂停执行。通过传递一个 Command,可以与来自人类的新输入一起恢复执行

interrupt 在人体工学上与 Python 的内置 input() 类似

将 human_assistance 工具添加到 chatbot 中。该工具使用 interrupt 接收来自人类的信息
'''

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langchain_tavily import TavilySearch

memory = InMemorySaver()

from dotenv import load_dotenv
load_dotenv()

# 构建state schema
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 将 human_assistance 工具添加到 chatbot 中。该工具使用 interrupt 接收来自人类的信息
@tool
def human_assistance(query: str) -> str:
    """
    Request assistance from a human.
    与 Python 的内置 input() 函数类似，在工具中调用 interrupt 将暂停执行。
    进度基于检查点持久化；因此，如果它使用 Postgres 进行持久化，只要数据库处于活动状态
    它就可以随时恢复。在这个例子中，它使用内存检查点进行持久化，只要 Python 内核正在运行，就可以随时恢复
    """
    human_response = interrupt({"query": query})
    return human_response["data"]

# 初始化工具
web_search = TavilySearch(max_results=2)
tools = [web_search, human_assistance]

# 初始化模型 绑定工具
llm = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

# 定义Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
# 增加Node
graph_builder.add_node("chatbot", chatbot)

# 定义工具Node
tool_node = ToolNode(tools)
# 增加工具Node
graph_builder.add_node("my_tools", tool_node)

# 增加条件边
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,    # langgraph自带的工具条件函数返回字符串是写死的，因此下面的映射字典，只能写tools和__end__
    {"tools": "my_tools", "__end__": END} # 如果不需要工具调用 直接结束
)
# 增加普通边
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("my_tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
#graph = graph_builder.compile()
# 使用提供的检查点编译 graph，该检查点将在 graph 遍历每个 Node 时将 State 作为检查点
graph = graph_builder.compile(checkpointer=memory)

In [6]:
user_input = "I need some expert guidance for building an AI agent. Could you request assistance for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I need some expert guidance for building an AI agent. Could you request assistance for me?
================================== Ai Message ==================================
Tool Calls:
  human_assistance (28bb5241-7295-4569-b13f-656e5f562488)
 Call ID: 28bb5241-7295-4569-b13f-656e5f562488
  Args:
    query: I need some expert guidance for building an AI agent.


In [ ]:
# 聊天机器人生成了一个工具调用，但随后执行被中断。如果你检查图状态，你会发现它停在工具节点上
snapshot = graph.get_state(config)
snapshot.next

('my_tools',)

In [8]:
human_response = (
    "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
    " It's much more reliable and extensible than simple autonomous agents."
)

human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (28bb5241-7295-4569-b13f-656e5f562488)
 Call ID: 28bb5241-7295-4569-b13f-656e5f562488
  Args:
    query: I need some expert guidance for building an AI agent.
================================= Tool Message =================================
Name: human_assistance

We, the experts are here to help! We'd recommend you check out LangGraph to build your agent. It's much more reliable and extensible than simple autonomous agents.
================================== Ai Message ==================================

The experts recommend using LangGraph to build your AI agent. They say it's more reliable and extensible than simple autonomous agents.
